## Amazon AgentCore Bedrock Code Interpreter を使用したエージェントベースのコード実行 - チュートリアル (LangChain)
このチュートリアルでは、Python を使用してコード実行により回答を検証する AI エージェントの作成方法を説明します。LLM によって生成されたコードを実行するために Amazon Bedrock AgentCore Code Interpreter を使用します。

このチュートリアルでは、AgentCore Bedrock Code Interpreter を使用して以下を行う方法を説明します：
1. サンドボックス環境のセットアップ
2. ユーザークエリに基づいてコードを生成する LangChain ベースのエージェントの設定
3. Code Interpreter を使用したサンドボックス環境でのコード実行
4. ユーザーへの結果表示

## 前提条件
- Bedrock AgentCore Code Interpreter へのアクセス権を持つ AWS アカウント
- Code Interpreter リソースを作成・管理するために必要な IAM 権限
- 必要な Python パッケージのインストール（boto3、bedrock-agentcore、LangChain を含む）
- IAM ロールには Amazon Bedrock でモデルを呼び出す権限が必要
 - US Oregon (us-west-2) リージョンの Claude 3.5 Sonnet モデルへのアクセス

## IAM 実行ロールには以下の IAM ポリシーをアタッチする必要があります



~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## 動作の仕組み

コード実行サンドボックスは、Code Interpreter、シェル、ファイルシステムを備えた隔離環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。Large Language Model がツール選択を支援した後、このセッション内でコードが実行され、その後ユーザーまたはエージェントに返されて統合されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、Code Interpreter セッションを初期化します。

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import code_session
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, tool
from langchain_aws import ChatBedrockConverse
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json


## 2. システムプロンプトの定義
AI アシスタントの動作と機能を定義します。回答は常にコード実行とデータに基づく推論によって検証するようアシスタントに指示します。

In [ ]:
SYSTEM_PROMPT = """あなたはコード実行を通じてすべての回答を検証する親切な AI アシスタントです。

検証の原則：
1. コード、アルゴリズム、計算について主張する場合は、それを検証するコードを書く
2. execute_python を使用して数学的計算、アルゴリズム、ロジックをテストする
3. 回答を出す前に理解を検証するテストスクリプトを作成する
4. 常に実際のコード実行で作業を示す
5. 不確かな場合は、制限を明示し、可能な範囲で検証する

アプローチ：
- プログラミングの概念について質問された場合、コードで実装して実演する
- 計算を求められた場合、プログラムで計算しコードを示す
- アルゴリズムを実装する場合、正確性を証明するテストケースを含める
- 透明性のために検証プロセスを文書化する
- サンドボックスは実行間で状態を維持するため、以前の結果を参照できる

利用可能なツール：
- execute_python: Python コードを実行して出力を確認

レスポンス形式：execute_python ツールは以下を含む JSON レスポンスを返します：
- sessionId: サンドボックスセッション ID
- id: リクエスト ID
- isError: エラーの有無を示すブール値
- content: type と text/data を含むコンテンツオブジェクトの配列
- structuredContent: コード実行の場合、stdout、stderr、exitCode、executionTime を含む"""

## 3. コード実行ツールの定義
次に、エージェントがコードサンドボックスでコードを実行するためのツールとして使用する関数を定義します。@tool デコレータを使用して、エージェントのカスタムツールとして関数をアノテーションします。

アクティブな Code Interpreter セッション内では、サポートされている言語（Python、JavaScript）でコードを実行し、依存関係の設定に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [ ]:
@tool
def execute_python(code: str, description: str = "") -> str:
    """サンドボックスで Python コードを実行します。"""
    
    if description:
        code = f"# {description}\n{code}"
    
    print(f"\n 生成されたコード: {code}")
    
    with code_session("us-west-2") as code_client:
        response = code_client.invoke("executeCode", {
            "code": code,
            "language": "python",
            "clearContext": False
        })
    
    for event in response["stream"]:
        return json.dumps(event["result"])

## 4. エージェントの設定
LangChain SDK を使用してエージェントを作成・設定します。上で定義したシステムプロンプトとコード生成実行用ツールを提供します。

#### 4.1 言語モデルの初期化

In [ ]:
llm = ChatBedrockConverse(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",region_name="us-west-2")

#### 4.2 プロンプトテンプレートの定義

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

#### 4.3 カスタムツールのリストを作成

In [ ]:
tools = [execute_python]

### 4.4 エージェントエグゼキュータの作成


In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 5. クエリの定義
コード実行機能を持つエージェントをテストするためのサンプルクエリを定義します。

In [ ]:
query="太陽系のすべての惑星は、地球と月の間に収まりますか？"

## 6. エージェントの呼び出しとレスポンス処理
クエリでエージェントを呼び出し、レスポンスを処理します。エージェントは仮説を立てることから推論を開始し、それをコードに変換して検証し、Code Interpreter でコードを実行することに注目してください。

In [ ]:
response=agent_executor.invoke({"input": query})

### エージェントからの最終レスポンス...

In [ ]:
print(response['output'][0]['text'])